In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, roc_auc_score
from sklearn.metrics import classification_report
import sklearn.metrics
from sklearn.ensemble import RandomForestClassifier



In [2]:
df = pd.read_csv("school-21-student-expulsion-prediction/train.csv")

In [3]:
df.head(5)

,id,Birth date,Native city,Gender,Wave id,Level,Heard about school from,Life status,day_00,day_01,...,rush_01,rush_02,exam_00,exam_01,exam_02,exam_final,Contract termination date,Memory entrance game,Logic entrance game,contract_status
0,129212391,1990-01-18,Пермь,male,3,7.463235,google_ads,work,5,NaN,...,0.0,0.0,75.0,43.0,100.0,44.0,NaN,9,11,1
1,566688420,1996-07-31,Волгоград,female,1,8.785714,NaN,NaN,0,NaN,...,0.0,0.0,0.0,16.0,22.0,22.0,NaN,7,10,1
2,242300495,1992-06-27,Новосибирск,female,2,6.775000,other,work,0,NaN,...,NaN,NaN,0.0,0.0,16.0,27.0,NaN,7,11,1
3,715424753,1996-06-15,Тула,female,1,8.146104,NaN,NaN,5,0.0,...,0.0,0.0,0.0,0.0,27.0,27.0,NaN,9,6,1
4,382884118,1968-07-18,Голицыно,male,3,4.272727,other,other,0,0.0,...,0.0,0.0,80.0,38.0,0.0,45.0,NaN,7,12,1


In [4]:
to_drop = ['id','Heard about school from','Contract termination date','Native city']

In [5]:
df = df.drop(to_drop, axis=1)

In [6]:
import datetime as dt

In [7]:
now = pd.to_datetime('now')

In [8]:
df['Birth date'] = pd.to_datetime(df['Birth date'], errors='coerce')

In [9]:
df['Birth date'] = (now.year - df['Birth date'].dt.year) - ((now.month - df['Birth date'].dt.month) < 0)

In [10]:
df = df.fillna(0)

In [11]:
df['Gender'].replace({'female':0,'male':1},inplace=True)

In [12]:
df

,Birth date,Gender,Wave id,Level,Life status,day_00,day_01,day_02,day_03,day_04,...,rush_00,rush_01,rush_02,exam_00,exam_01,exam_02,exam_final,Memory entrance game,Logic entrance game,contract_status
0,30,1,3,7.463235,work,5,0.0,0.0,80.0,0.0,...,0.0,0.0,0.0,75.0,43.0,100.0,44.0,9,11,1
1,23,0,1,8.785714,0,0,0.0,10.0,45.0,5.0,...,110.0,0.0,0.0,0.0,16.0,22.0,22.0,7,10,1
2,27,0,2,6.775000,work,0,0.0,20.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,27.0,7,11,1
3,23,0,1,8.146104,0,5,0.0,35.0,25.0,20.0,...,0.0,0.0,0.0,0.0,0.0,27.0,27.0,9,6,1
4,51,1,3,4.272727,other,0,0.0,0.0,5.0,85.0,...,120.0,0.0,0.0,80.0,38.0,0.0,45.0,7,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,24,1,3,4.149733,study,10,0.0,0.0,10.0,20.0,...,0.0,0.0,0.0,0.0,0.0,22.0,18.0,7,11,1
1056,21,1,3,4.320856,study,5,0.0,10.0,0.0,70.0,...,120.0,0.0,0.0,20.0,22.0,32.0,27.0,11,12,1
1057,23,1,2,6.675000,study,30,30.0,10.0,65.0,70.0,...,0.0,96.0,0.0,100.0,48.0,80.0,45.0,8,16,1
1058,28,1,3,6.483333,0,50,0.0,80.0,30.0,47.0,...,105.0,0.0,0.0,0.0,64.0,64.0,45.0,8,12,1


In [13]:
df

,Birth date,Gender,Wave id,Level,Life status,day_00,day_01,day_02,day_03,day_04,...,rush_00,rush_01,rush_02,exam_00,exam_01,exam_02,exam_final,Memory entrance game,Logic entrance game,contract_status
0,30,1,3,7.463235,work,5,0.0,0.0,80.0,0.0,...,0.0,0.0,0.0,75.0,43.0,100.0,44.0,9,11,1
1,23,0,1,8.785714,0,0,0.0,10.0,45.0,5.0,...,110.0,0.0,0.0,0.0,16.0,22.0,22.0,7,10,1
2,27,0,2,6.775000,work,0,0.0,20.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,27.0,7,11,1
3,23,0,1,8.146104,0,5,0.0,35.0,25.0,20.0,...,0.0,0.0,0.0,0.0,0.0,27.0,27.0,9,6,1
4,51,1,3,4.272727,other,0,0.0,0.0,5.0,85.0,...,120.0,0.0,0.0,80.0,38.0,0.0,45.0,7,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,24,1,3,4.149733,study,10,0.0,0.0,10.0,20.0,...,0.0,0.0,0.0,0.0,0.0,22.0,18.0,7,11,1
1056,21,1,3,4.320856,study,5,0.0,10.0,0.0,70.0,...,120.0,0.0,0.0,20.0,22.0,32.0,27.0,11,12,1
1057,23,1,2,6.675000,study,30,30.0,10.0,65.0,70.0,...,0.0,96.0,0.0,100.0,48.0,80.0,45.0,8,16,1
1058,28,1,3,6.483333,0,50,0.0,80.0,30.0,47.0,...,105.0,0.0,0.0,0.0,64.0,64.0,45.0,8,12,1


In [14]:
df['Life status'].replace({'work':1,'study':2,'other':0},inplace=True)

In [15]:
df

,Birth date,Gender,Wave id,Level,Life status,day_00,day_01,day_02,day_03,day_04,...,rush_00,rush_01,rush_02,exam_00,exam_01,exam_02,exam_final,Memory entrance game,Logic entrance game,contract_status
0,30,1,3,7.463235,1,5,0.0,0.0,80.0,0.0,...,0.0,0.0,0.0,75.0,43.0,100.0,44.0,9,11,1
1,23,0,1,8.785714,0,0,0.0,10.0,45.0,5.0,...,110.0,0.0,0.0,0.0,16.0,22.0,22.0,7,10,1
2,27,0,2,6.775000,1,0,0.0,20.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,16.0,27.0,7,11,1
3,23,0,1,8.146104,0,5,0.0,35.0,25.0,20.0,...,0.0,0.0,0.0,0.0,0.0,27.0,27.0,9,6,1
4,51,1,3,4.272727,0,0,0.0,0.0,5.0,85.0,...,120.0,0.0,0.0,80.0,38.0,0.0,45.0,7,12,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1055,24,1,3,4.149733,2,10,0.0,0.0,10.0,20.0,...,0.0,0.0,0.0,0.0,0.0,22.0,18.0,7,11,1
1056,21,1,3,4.320856,2,5,0.0,10.0,0.0,70.0,...,120.0,0.0,0.0,20.0,22.0,32.0,27.0,11,12,1
1057,23,1,2,6.675000,2,30,30.0,10.0,65.0,70.0,...,0.0,96.0,0.0,100.0,48.0,80.0,45.0,8,16,1
1058,28,1,3,6.483333,0,50,0.0,80.0,30.0,47.0,...,105.0,0.0,0.0,0.0,64.0,64.0,45.0,8,12,1


In [16]:
df.to_csv('data/after_clean.csv')

In [17]:
corr = df.corr()

In [18]:
corr.to_excel("data/correlation.xlsx")

In [19]:
y = df.contract_status

In [20]:
X = df.drop('contract_status',axis=1)

In [21]:
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.3, random_state=1)

In [22]:
clf = RandomForestClassifier(random_state=47,n_jobs=-1,n_estimators=200,class_weight='balanced_subsample')

In [24]:
param_destributions = {'max_features': list(range(1,X.shape[1])), "max_depth": list(range(5,30))}

In [26]:
from sklearn import metrics
clf.fit(X_train,y_train)
y_pred = clf.predict(X_test)
print("Результат на тестовых данных(модель по умолчанию): %f " % (100*metrics.balanced_accuracy_score(y_test,y_pred)))

Результат на тестовых данных(модель по умолчанию): 56.064815 


In [29]:
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(clf,param_distributions=param_destributions,n_iter=50,cv=5,scoring="balanced_accuracy",n_jobs=-1)
random_search.fit(X_train,y_train)

RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight='balanced_subsample',
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,

In [32]:
# Сохраняем оптимальную модель и смотрим на ее параметры:
model = random_search.best_estimator_
print("Оптимальные параметры: %s , оценка гиперпараметров: %0.2f " % (random_search.best_params_, random_search.best_score_))
# ограничение параметра max_depth позволяет избежать переобучения, и потому дает лучшие результаты)

Оптимальные параметры: {'max_features': 15, 'max_depth': 5} , оценка гиперпараметров: 0.76 


In [35]:
import pickle
filename = "my_model_RF.sav"
pickle.dump(model,open(filename,'wb'))

In [36]:
model = pickle.load(open(filename, 'rb'))

In [37]:
y_pred = model.predict(X_test)

In [38]:
print("Результат на тестовых данных: %f " %(100*metrics.balanced_accuracy_score(y_test,y_pred)))

Результат на тестовых данных: 78.101852 
